In [0]:
import tensorflow as tf, pandas as pd, numpy as np, time, os

In [0]:
!pip install pydub

In [0]:
# pydub
from pydub import AudioSegment

In [0]:
!git clone https://github.com/RedbirdTaiwan/silic.git

Cloning into 'silic'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 35 (delta 4), reused 16 (delta 3), pack-reused 18
Unpacking objects: 100% (35/35), done.


In [0]:
!ls silic/
!ls silic/model
!ls silic/sample

LICENSE  model	README.md  sample  scripts
conv_labels.txt  frozen_graph.pb  sound_class.txt
11794.mp3


In [0]:
class Autolabel():
  def __init__(self, label_file, graph_file):
    self.label_file = label_file
    self.graph_file = graph_file
    self.__sound = ''
    self.results = {}

  def audiofile(self, audio_file):
    frame_rate = 30000
    filext = audio_file[-3:].lower()
    if filext == "mp3":
        self.__sound = AudioSegment.from_mp3(audio_file)
    elif filext == "wma":
        self.__sound = AudioSegment.from_file(audio_file, "wma")
    elif filext == "m4a":
        self.__sound = AudioSegment.from_file(audio_file, "m4a")
    elif filext == "ogg":
        self.__sound = AudioSegment.from_ogg(audio_file)
    else:
        self.__sound = AudioSegment.from_wav(audio_file)
    if self.__sound.frame_rate > frame_rate:
        self.__sound = self.__sound.set_frame_rate(frame_rate)
    if self.__sound.channels > 1:
        self.__sound = self.__sound.split_to_mono()[0]
    if not self.__sound.sample_width == 2:
        self.__sound = self.__sound.set_sample_width(2)

  def ailabel(self, **kwargs):
    if not len(self.__sound):
      print('use ".audiofile(<audio_file_path>)" to import a recording')
      return false
    if not 'step' in kwargs.keys():
      step = 3000
    else:
      step = kwargs['step']
    sample_rate = 30000
    clip_duration_ms = 3000
    labels_list = [line.rstrip() for line in tf.io.gfile.GFile(self.label_file)]
    with tf.io.gfile.GFile(self.graph_file, 'rb') as f:
      graph_def = tf.compat.v1.GraphDef()
      graph_def.ParseFromString(f.read())
      tf.import_graph_def(graph_def, name='')
    sess = tf.compat.v1.Session()
    softmax_tensor = sess.graph.get_tensor_by_name('labels_softmax:0')
    self.results = {k:[] for k in labels_list[2:]}
    for t in range(0,len(self.__sound),step):
      if (t + clip_duration_ms) < len(self.__sound):
        soundclip = self.__sound[t:t+clip_duration_ms]
        soundclip.export('tmp.wav', format="wav")
        with open('tmp.wav', 'rb') as wav:
          wav_data2 = wav.read()
        predictions, = sess.run(softmax_tensor, {'wav_data:0': wav_data2})
        for node_id in range(2, len(labels_list)):
          song_of_bird = labels_list[node_id]
          score = round(predictions[node_id],3)
          self.results[song_of_bird].append([t,score])

In [0]:
test = Autolabel('silic/model/conv_labels.txt', 'silic/model/frozen_graph.pb')

In [0]:
test.audiofile('silic/sample/11794.mp3')

In [0]:
test.ailabel(step=1000)

In [37]:
results = test.results
soundclass = pd.read_csv('silic/model/sound_class.txt', index_col=0, sep='\t').T.to_dict()
for soundid, scores in zip(results.keys(), results.values()):
  for score in scores:
    if score[1] >= 0.8:
      sound = soundclass[int(soundid)]['scientific_name'] + ':' + soundclass[int(soundid)]['class_name']
      time_start = int(score[0]/1000)
      ai_score = score[1]
      print(sound, time_start, ai_score)

Glaucidium brodiei:S-01 7 0.935
Glaucidium brodiei:S-01 9 0.953
Glaucidium brodiei:S-01 10 0.916
Glaucidium brodiei:S-01 11 0.929
Glaucidium brodiei:S-01 13 0.82
Glaucidium brodiei:S-01 14 0.929
Glaucidium brodiei:S-01 226 0.81
Glaucidium brodiei:S-01 229 0.82
Glaucidium brodiei:S-01 249 0.906
Glaucidium brodiei:S-01 261 0.959
Glaucidium brodiei:S-01 262 0.93
Glaucidium brodiei:S-01 263 0.85
Glaucidium brodiei:S-01 281 0.848
Glaucidium brodiei:S-01 282 0.885
Glaucidium brodiei:S-01 286 0.885
Glaucidium brodiei:S-01 287 0.827
Ninox japonica:S-01 0 0.907
Ninox japonica:S-01 1 0.938
Ninox japonica:S-01 35 0.982
Ninox japonica:S-01 36 0.994
Ninox japonica:S-01 37 0.971
Ninox japonica:S-01 38 0.999
Ninox japonica:S-01 39 0.99
Ninox japonica:S-01 40 0.992
Ninox japonica:S-01 41 0.965
Ninox japonica:S-01 42 0.999
Ninox japonica:S-01 43 0.992
Ninox japonica:S-01 44 0.955
Ninox japonica:S-01 45 0.982
Ninox japonica:S-01 46 0.994
Ninox japonica:S-01 47 0.966
Ninox japonica:S-01 48 0.91
Ninox jap